# Deconvolution under Gaussian prior

We use an Expectation-Maximization framework in order to apply the $MAP_k$ algorithm with a Gaussian prior.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [24]:
n = 5 # Image size is n*n
N = n**2 # Pixels per image
K = 1 # Kernel size is (2K+1)*(2K+1)
# (note that when performing convolution, we consider the image to be its own symmetric past the edges,
# with the boundary pixels taking up two spaces)

eta = 0.01
sigma = 0.1

## EM algorithm

Alternate between two steps : estimating the likely image (and its covariance), and estimating the likely kernel.

### E-step
Set $$q(x) = p(x \mid y,k),$$
and compute its mean $\mu$ and covariance $C$ (i.e. estimate the likely image, but also look at how much it could vary).

Under a Gaussian prior, $\mu$ and $C$ can be expressed simply in terms of $A_x$ and $b_x$:
$$ A_x = \frac{1}{\eta^2}T_k^t T_k + \frac{1}{\sigma^2}(T_{f_v}^t T_{f_v} + T_{f_h}^t T_{f_h})$$
$$ b_x = \frac{1}{\eta^2} T_k^t y $$
Here $T_k$, $T_{f_h}$ and $T_{f_v}$ are the convolution operators.

The conditional distribution $q$ is in fact Gaussian as well, and we can simply express its features given $A_x$ and $b_x$:
$$ C = A_x^{-1} $$
$$ \mu = C b_x $$

In [31]:
# Re-indexing in numpy functions by default with 'C'-order, with the last index changing fastest.

# Image indices go from (0,0) to (N-1,N-1) included
image_index = np.reshape(np.arange(N),(n,n))

# Kernel indices go from (-K,-K) to (K,K) included
def kernel_index(u,v):
    return (u+K)*(2*K+1) + v+K

# TODO : When performing convolution, we could consider the image to be its own symmetric past the edges,
# with the boundary pixels taking up two spaces. ???
# Actual implementation : return an image with zero values ???
def Toeplitz_matrix(kernel):
    T = np.zeros((N,N))
    for i1 in range(n):
        for j1 in range(n):
            for i2 in range(n):
                for j2 in range(n):
                    if -K <= i2-i1 <= K and -K <= j2-j1 <= K:
                        #print(i1,j1,i2,j2)
                        T[image_index[i1,j1],image_index[i2,j2]] = kernel[kernel_index(i2-i1,j2-j1)]
    return T

kernel = np.arange((2*K+1)**2)
#print(Toeplitz_matrix(kernel))

# fh of fv matrix computation
f1_matrix = np.zeros((N,N))
for i in range(n):
    for j in range(n-1):
        f1_matrix[image_index[i,j],image_index[i,j+1]] = 1
        f1_matrix[image_index[i,j],image_index[i,j]] = -1
f2_matrix = np.zeros((N,N))
for i in range(n-1):
    for j in range(n):
        f2_matrix[image_index[i,j],image_index[i+1,j]] = 1
        f2_matrix[image_index[i,j],image_index[i,j]] = -1

### M-step
Estimate the kernel $k$ minimizing
$$ E_q \left[  || k*x - y ||^2  \right] .$$
Since we are considering the $q$-expectation of something quadratic, it only depends on the first two moments of $q(x)$, so $\mu$ and $C$ are sufficient.